In [ ]:
from matplotlib.pyplot import axis, plot
import numpy as np
np.set_printoptions(precision=3, suppress=True, threshold=10)

# Curve Fitting

## Linear polynomial fitting

In [ ]:
from numpy import polyfit, poly1d, pi

In [ ]:
x = np.linspace(-5, 5, 100)
y = 4 * x + 1.5
noisy_y = y + np.random.randn(y.shape[-1]) * 2.5

In [ ]:
p = plot(x, noisy_y, 'rx')
p = plot(x, y, 'b:')

In [ ]:
coefficients = polyfit(x, noisy_y, 1)
print(coefficients)

In [ ]:
p = plot(x, noisy_y, 'rx')
p = plot(x, coefficients[0] * x + coefficients[1], 'k-')
p = plot(x, y, 'b--')

In [ ]:
f = poly1d(coefficients)
p = plot(x, noisy_y, 'rx')
p = plot(x, f(x))

In [ ]:
f

In [ ]:
print(f + 2 * f **2)

## Sine curve example

In [ ]:
x = np.linspace(-np.pi, np.pi, 100)
y = np.sin(x)

In [ ]:
y1 = poly1d(polyfit(x, y, 1))
y3 = poly1d(polyfit(x, y, 3))
y5 = poly1d(polyfit(x, y, 5))
y7 = poly1d(polyfit(x, y, 7))
y9 = poly1d(polyfit(x, y, 9))

In [ ]:
x = np.linspace(-3 * np.pi, 3 * np.pi, 100)
p = plot(x, np.sin(x), 'k')
p = plot(x, y1(x))
p = plot(x, y3(x))
p = plot(x, y5(x))
p = plot(x, y7(x))
p = plot(x, y9(x))
a = axis([-3 * np.pi, 3 * np.pi, -1.25, 1.25])

## Least-Squares Fitting

In [ ]:
from scipy.linalg import lstsq
from scipy.stats import linregress
from scipy.optimize import leastsq

In [ ]:
x = np.linspace(0, 5, 100)
y = 0.5 * x + np.random.randn(x.shape[-1]) * 0.35
p = plot(x,y,'x')

$$XC = Y$$

where for a polynomial of order $N$ and $M$ observations

$$\left[ \begin{matrix}
x_0^{N-1} & \dots & x_0 & 1 \\\
x_1^{N-1} & \dots & x_1 & 1 \\\
\dots & \dots & \dots & \dots \\\
x_M^{N-1} & \dots & x_M & 1
\end{matrix}\right] 
\left[ \begin{matrix} C_{N-1} \\\ \dots \\\ C_1 \\\ C_0 \end{matrix} \right] =
\left[ \begin{matrix} y_0 \\\ y_1 \\\ \dots \\\ y_M \end{matrix} \right]$$

### Scipy.linalg.lstsq 

In [ ]:
X = np.hstack((x[:, np.newaxis], np.ones((x.shape[-1],1))))
X

In [ ]:
C, resid, rank, s = lstsq(X, y)
p = plot(x, y, 'rx')
p = plot(x, C[0] * x + C[1], 'k--')
print("sum squared residual = {:.3f}".format(resid))
print("rank of the X matrix = {}".format(rank))
print("singular values of X = {}".format(s))

### Scipy.stats.linregress

In [ ]:
slope, intercept, r_value, p_value, stderr = linregress(x, y)

In [ ]:
p = plot(x, y, 'rx')
p = plot(x, slope * x + intercept, 'k--')
print("R-value = {:.3f}".format(r_value))
print("p-value (probability there is no correlation) = {:.3e}".format(p_value))
print("Root mean squared error (𝜎) of the fit = {:.3f}".format(np.sqrt(stderr)))

## Curve fitting with callables

$y = a e^{-b sin( f x + \phi)}$

First, let's generate a function with some parameters and make noisy data from it.  We'll try a couple of methods to see if we can recover the original parameters from the noisy data.

In [ ]:
def function(x, a , b, f, phi):
    """a function of x with four parameters"""
    result = a * np.exp(-b * np.sin(f * x + phi))
    return result

In [ ]:
x = np.linspace(0, 2 * pi, 50)
actual_parameters = [3, 2, 1.25, pi / 4]
y = function(x, *actual_parameters)
p = plot(x,y)

In [ ]:
from scipy.stats import norm
y_noisy = y + 0.8 * norm.rvs(size=len(x))
p = plot(x, y, 'k-')
p = plot(x, y_noisy, 'rx')

### scipy.optimize.leastsq

In [ ]:
def f_err(p, y, x):
    """Returns the error between y and function(x) for a set of x-y points"""
    return y - function(x, *p)

In [ ]:
c, ret_val = leastsq(f_err, [1, 1, 1, 1], args=(y_noisy, x))

If `ret_val` is 1, 2, 3, 4, then the solution was found, and execution was successful.

In [ ]:
p = plot(x, y_noisy, 'rx')
p = plot(x, function(x, *c), 'k--')
print(f"ret_val={ret_val}")
print("Actual parameters   ={}.".format(", ".join("{:5.2f}".format(p) for p in actual_parameters)))
print("Scipy Least Squares ={}.".format(", ".join("{:5.2f}".format(p) for p in c)))

Initial guess makes a difference:

In [ ]:
c, ret_val = leastsq(f_err, [0, 0, 0, 0], args=(y_noisy, x))
p = plot(x, y, 'rx')
p = plot(x, function(x, *c), 'k--')
print(f"ret_val={ret_val}")
print("Actual parameters   ={}.".format(", ".join("{:5.2f}".format(p) for p in actual_parameters)))
print("Scipy Least Squares ={}.".format(", ".join("{:5.2f}".format(p) for p in c)))

### scipy.optimize.curve_fit

General purpose curve fitting interface to Fortran `MINPACK`

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
p_est, err_est = curve_fit(function, x, y_noisy)

In [ ]:
p = plot(x, y_noisy, "rx")
p = plot(x, function(x, *p_est), "k--")
print("Actual parameters   ={}.".format(", ".join("{:5.2f}".format(p) for p in actual_parameters)))
print("Scipy Least Squares ={}.".format(", ".join("{:5.2f}".format(p) for p in p_est)))

In [ ]:
# err_est is a covariance matrix of the estimates
# The diagonals are the variances of the individual parameters.
print(err_est)

In [ ]:
print("normalized relative errors for each parameter")
print("   a\t  b\t f\tphi")
print(np.sqrt(err_est.diagonal()) / p_est)